### Projeto de Embedding aplicado

O objetivo aqui é tentar colocar em uso o que foi aprendido no projeto [de embedding](./Tensorflow_Embedding.ipynb).
Aqui temos como objetivo:

    1 - Carregar uma base de dados real de mensagens.
    2 - Aplicar o embedding multilingual na base de dados.
    3 - Aplicar K-MEANS e Propagação por Afinidade nos vetores gerados. 
    4 - Gerar CSV do resultado

[*: projetor do tensorflow](http://projector.tensorflow.org/)

Query:
```
union customEvents
| where customDimensions ["ChatbotId"] == "cb4683478" and customDimensions["IntentName"] == "unknown" and name == "MessageIn" and customDimensions ["InAttendance"] != true
| project text = parse_json(tostring(customDimensions["Message"])).text
```

In [1]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

#### Carregando e limpando os dados

Aqui em baixo faço uma limpeza bem básica que tem mais haver com a formatação do arquivo. Não preciso fazer mais que isso porque o modelo usa o tensorflow_text para tratar o resto.

In [12]:
data = []
with open("./datasets/testando.csv", encoding="utf-8") as file:
    data = [line for line in file if not line.isnumeric()]
data = data[1:]
data = [phrase.replace("\n","") for phrase in data]
data = [phrase.replace("\"","") for phrase in data]
data = [phrase for phrase in data if len(phrase) != 0]
data = [phrase for phrase in data if not phrase.isnumeric()]
data[:10],len(data)

(['113.882.181-00',
  '?',
  'Sobri o meu passe card',
  'Por gentileza ????',
  '????',
  'Ok',
  'Cadê',
  '??????',
  'Queria a foto da cortina de Led',
  'E os valores'],
 12238)

In [26]:
import tensorflow as tf
from IPython.display import clear_output


def to_embed_vector(data,embed,batch = 10, i = 0, size = None):
    if size > len(data) or size is None:
        size = len(data)
    output = None
    while i < size:
        if i == 0:
            output = embed(data[i:i+batch])
        else:
            if i+batch < size:
                output = tf.concat([output, embed(data[i:i+batch])], 0)
            else:
                output = tf.concat([output, embed(data[i:size])], 0)          
        i+=batch
        print("{0}/{1}".format(i,size))
        clear_output(wait=True)
    return output

output = to_embed_vector(data, embed,size=1000)

1000/1000


In [27]:
output.shape

TensorShape([1000, 512])

### Estratégias para relatório

In [34]:
import nltk
def most_frequent_words(class_sample, cluster, n=3):
    #ps = nltk.stem.RSLPStemmer()
    phrases = [data[i] for i in range(len(cluster.labels_)) if cluster.labels_[i] == class_sample]
    frequency = {}
    for phrase in phrases:
        tokens = nltk.word_tokenize(phrase)
        for token in tokens: 
            token = token.lower()
            if token in nltk.corpus.stopwords.words("portuguese"):
                continue
            if not token.isalpha():
                continue
            if token in frequency.keys():
                frequency[token] += 1
            else:
                frequency[token] = 1
    frequency = sorted(frequency.items(), key = lambda item: item[1], reverse=True)
    return [word for word, count in frequency[:n]]

def nearest_words(output, embed, class_sample, cluster, n=5):
    words = most_frequent_words(class_sample, cluster, n=n)
    indices = [i for i in range(len(cluster.labels_)) if cluster.labels_[i] == class_sample]
    dist = np.inf
    nearest_word = None
    for word in words:
        mean = 0
        for i in indices:
            mean += np.linalg.norm(output[i].numpy() - embed(word).numpy())
        mean /= len(indices)
        if mean < dist:
            nearest_word = word
            dist = mean
    return "" if nearest_word is None else nearest_word

def show_first_n(class_sample,cluster,n=10, nearest=None, most_frequent=None):
    result = "\n".join([data[i] + "\t"+ str(cluster.labels_[i]) + "\t" + nearest + "\t" + most_frequent for i in range(len(cluster.labels_)) if cluster.labels_[i] == class_sample][:n])
    return result

#### Aplicando KMEANS

Aqui aplico o [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) para clusterização das frases selecionadas. Roda bem rápido.

In [29]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20,random_state=0).fit(output.numpy())

Abaixo, uma breve representação de como os metadados estão configurados.

In [30]:
def save_tesor(tensor, metadata):
    output = ""
    tensor = tensor.numpy()
    for i in range(tensor.shape[0]):
        for j in range(tensor.shape[1]):
            output += str(tensor[i,j]) + "\t"
        output += "\n"
    with open("tensor.tsv", "w", encoding="utf-8") as file:
        file.write(output)
    with open("metadata.tsv", "w", encoding="utf-8") as file:
        file.write("\n".join(["\t".join(row) for row in metadata]))
        
save_tesor(output, [["phrases", "colors"]] + [[data[i],str(kmeans.labels_[i])] for i in range(len(kmeans.labels_))])

In [31]:
[["phrases", "colors"]] + [[data[i],kmeans.labels_[i]] for i in range(len(kmeans.labels_))][:10]

[['phrases', 'colors'],
 ['113.882.181-00', 15],
 ['?', 10],
 ['Sobri o meu passe card', 16],
 ['Por gentileza ????', 10],
 ['????', 10],
 ['Ok', 3],
 ['Cadê', 13],
 ['??????', 10],
 ['Queria a foto da cortina de Led', 6],
 ['E os valores', 8]]

In [32]:
index = 10
print(most_frequent_words(index,kmeans, n=5))
nearest_words(output,embed,index,kmeans, n=5)

['gentileza', 'blz', 'poderia', 'ajudar', 'alguém']


'blz'

#### Clusters encontrados. 

In [35]:
for i in set(kmeans.labels_):
    most_freq = most_frequent_words(i,kmeans)
    nearest = nearest_words(output,embed,i,kmeans)
    print(show_first_n(i,kmeans, nearest=nearest, most_frequent=",".join(most_freq)))
    print("******************************************************************")

Mas a nota não abre	0	ainda	ainda,internet,sistema
Pneu passe carde foi perdido	0	ainda	ainda,internet,sistema
Ao acessar o Sirius está dando sistema inacessível.	0	ainda	ainda,internet,sistema
Discupa foi errado	0	ainda	ainda,internet,sistema
Solicitei uma nota fiscal de uma consulta que fiz e ainda não foi enviada	0	ainda	ainda,internet,sistema
Ainda não encerrou o atendimento	0	ainda	ainda,internet,sistema
voce nao mi passou po sac	0	ainda	ainda,internet,sistema
Já verificou o e-mail, e nada	0	ainda	ainda,internet,sistema
Minha internet está muito lenta	0	ainda	ainda,internet,sistema
Sistema não está enviando o primeiro boleto	0	ainda	ainda,internet,sistema
******************************************************************
Blz	1	oii	oii,ai,boa
Oii	1	oii	oii,ai,boa
Oii	1	oii	oii,ai,boa
❤️	1	oii	oii,ai,boa
❤️	1	oii	oii,ai,boa
Oii	1	oii	oii,ai,boa
So	1	oii	oii,ai,boa
Oii	1	oii	oii,ai,boa
Oie	1	oii	oii,ai,boa
oii	1	oii	oii,ai,boa
*********************************************************

#### Aplicando outra estratégia

Aqui é aplicado a [Afinidade de Propagação](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html#sklearn.cluster.AffinityPropagation). Essa estratégia de clusterização tem como hiperparâmetro o damping. Ela "descobre" um número de clusters existentes calibrado segundo o hiperparâmetro.

In [18]:
from sklearn.cluster import AffinityPropagation
ap = AffinityPropagation(random_state=5, verbose=True,damping=.9, max_iter=1000).fit(output.numpy()) #.775

KeyboardInterrupt: 

In [24]:
for i in set(ap.labels_):
    most_freq = most_frequent_words(i,ap)
    nearest = nearest_words(output,embed,i,ap)
    print(show_first_n(i,ap, nearest=nearest, most_frequent=",".join(most_freq)))
    print("******************************************************************")

Olá, gostaria de acompanhar on line os cursos da Professora Lucia Helena.	0	professora	helena,gostaria,lucia
Onde posso ver e ouvir os cursos da Prof.a Lucia Helena?	0	professora	helena,gostaria,lucia
Gostaria muito de participar  do curso , que a professora Lucia  Helena  vai dar , sobre os clássicos  indianos?	0	professora	helena,gostaria,lucia
Olá! Por favor gostaria de informações sobre palestras da Prof Lucia Helena	0	professora	helena,gostaria,lucia
Olá! Acabei de assistir a uma palestra da Prof. Lucia Helena Galvão no YouTube, gostei e comprei um Plano anual via QR CODE, como acesso?	0	professora	helena,gostaria,lucia
Gostaria de saber qual editor do Caibalion que a professora Lucia prefere pois quero comprar e tb o que tem os comentários dela.	0	professora	helena,gostaria,lucia
Assim que possivel, gostaria de informações sobre o Curso de Filosofia à Maneira Clássica e cursos com a Professora Lucia Helena	0	professora	helena,gostaria,lucia
Olá! Gostaria de comprar o curso da pro

In [21]:
save_tesor(output, [["phrases", "colors"]] + [[data[i],str(ap.labels_[i])] for i in range(len(ap.labels_))])

NameError: name 'save_tesor' is not defined

In [22]:
from tqdm import tqdm
def save_csv(filename,cluster):
    result = ""
    for i in tqdm(set(cluster.labels_)):
        most_freq = most_frequent_words(i,ap)
        nearest = nearest_words(output,embed,i,ap)
        result += show_first_n(i,cluster,n=100,nearest=nearest,most_frequent=",".join(most_freq)) + "\n"
    with open(filename+".tsv", "w", encoding="utf-8") as file:
        file.write("Frase\tGrupo\tPalavra Em Destaque No Grupo\tPalavras Frequentes No Grupo\n")
        file.write(result)

save_csv("teste",ap)

100%|████████████████████████████████████████████████████████████████████████████████| 118/118 [03:08<00:00,  1.60s/it]
